[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/05-RGB_compression/RGB_compression.ipynb)

# RGB image compression

Insights on compressing color images with PNG in the RGB domain.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
#!pwd
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/midtread_quantizer.py .
!ln -sf ~/quantization/midrise_quantizer.py .
!ln -sf ~/quantization/companded_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ../common.py .
import deadzone_quantizer as deadzone
import midtread_quantizer as midtread
import midrise_quantizer as midrise
import companded_quantizer as companded
import distortion
import image_3 as RGB_image
import image_1 as gray_image
import colored
import common
import information

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
#fn = home + "/MRVC/sequences/stockholm/"
#fn = home + "/MRVC/sequences/Hommer/"
#fn = home + "/MRVC/sequences/fluorescence/"
#fn = home + "/MRVC/sequences/blue_rose/"
#fn = home + "/MRVC/sequences/lena_color/"
fn = home + "/MRVC/sequences/nored_lena/"

components = ['R', 'G', 'B']

#quantizer = midtread
quantizer = deadzone
#quantizer = midrise
#Q_steps = range(128, 0, -1) # Maybe it does not make sense in digital quantization (generate loops)
Q_steps = [2**i for i in range(7, -1, -1)] # Quantization steps (simulating bit-plane encoding)

#quantizer = companded
#Q_steps = [2**i for i in range(16, -1, -1)] # Quantization steps (simulating bit-plane encoding)

print(Q_steps)

Notice that non embbeded quatization (using steps thar are different from a power of 2) steps (can produce loops in the RD curves due to the non-linearity of the integer division performed in the quantization).

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(np.int16) # 8 bits/components is not sufficient to shift the components to [-128, 127]
print(RGB_img.dtype)
print(RGB_img.max(), RGB_img.min())
common.show(RGB_img, fn + "000.png")

## Energy of the RGB components

In [ ]:
R_avg_energy = information.average_energy(RGB_img[...,0])
G_avg_energy = information.average_energy(RGB_img[...,1])
B_avg_energy = information.average_energy(RGB_img[...,2])
print(f"Average energy R = {R_avg_energy}")
print(f"Average energy G = {G_avg_energy}")
print(f"Average energy B = {B_avg_energy}")
total_RGB_avg_energy = R_avg_energy + G_avg_energy + B_avg_energy
print(f"Total RGB average energy (computed by adding the energies of the components) = {total_RGB_avg_energy}")
print(f"Total RGB average energy (computed directly from the RGB image) = {information.average_energy(RGB_img)*3}")

The RGB domain is additive. This basically tell us that the addition of the distortion generated in each component equals the distortion measured in the RGB domain (this is a trivial conclusion!!).

## Show some quantizations

In [ ]:
QS = 128 # Quantization Step
RGB_img = RGB_img.astype(np.int16) - 128 # Quantized data must be centered at 0
y, k = quantizer.quan_dequan(RGB_img, QS)
print("Used quantization indexes:", np.unique(k))
common.show(k, f"{quantizer.name} $\\Delta={QS}$")

In [ ]:
QS = 64 # Quantization Step
RGB_img = RGB_img.astype(np.int16) - 128 # Quantized data must be centered at 0
y, k = quantizer.quan_dequan(RGB_img, QS)
print("Used quantization indexes:", np.unique(k))
common.show(k, f"{quantizer.name} $\\Delta={QS}$")

## RD curve using same $\Delta$ for each RGB channel ($\Delta_{\text{R}} = \Delta_{\text{G}} = \Delta_{\text{B}}$)
To see the contribution of each channel to the RD curve.

In [ ]:
def same_delta_RD_curve(RGB_img, Q_steps, quantizer):
    RGB_img -= 128
    points = []
    for Q_step in Q_steps:
        y, k = quantizer.quan_dequan(RGB_img, Q_step)
        k = (k + 128).astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        rate = common.bits_per_color_pixel(k, str(Q_step) + '_') # Remember that k has 3 components
        _distortion = distortion.RMSE(RGB_img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

#same_delta_RD_points = same_delta_RD_curve(img - 128, Q_steps, quantizer)
same_delta_RD_points = same_delta_RD_curve(RGB_img, Q_steps, quantizer)

In [ ]:
print(quantizer.name)
with open(f"RGB_same_{quantizer.name}_RD_points.txt", 'w') as f:
    for item in same_delta_RD_points: #RD_points_sorted_by_slopes:
        f.write(f"{item[0]}\t{item[1]}\n")

## Generation of the "optimal" RD curve

In principle, if we take into account the RD curves of each of the components, we can use an optimal quantization pattern (three elements, one per component) to find the best progression of quantization patterns. However, since this pattern is applied directly to the RGB domain and each component contributes the same gain to the energy of its image as a whole, the only thing we are going to achieve is to find intermediate RD points (which belong to the convex-hull RD) between the points where it is verified that $ \ Delta_R = \ Delta_G = | Delta_B $, generated by quantization patterns where any of these equalities is not true.

In the case where the gain of the components is not identical, then we could have a progression of quantization patterns better than that represented by fn = home + "/ MRVC / sequences / lena_color /" (

In general, the contribution of the components to the quality of the reconstruction can be different because even being the distortion additive, the compression ratio of each component can be different.


. This contribution is expressed in terms of RD, "selecting", for each QS, first the component that has the highest RD slope. Thus, in general (in this case for all cases), the "optimal" RD curve using the following procedure:

1. The RD curve of each RGB channel is computed, for a number of quantization steps.
2. The RD points are sorted by slopes.

Notice that must be true that the RGB components are additive (colors are obtained by a linear combination of Red, Green, and Blue, i.e., if we increment the quality of one component, the quality of the reconstruction is increases in such increment also). Otherwise, the distortion must be computed in the RGB domain (considering the three components at the same time, that is the average among the components).

### RD curves of each RGB channel

In [ ]:
def RD_curve(RGB_img, Q_steps, component, components, quantizer):
    RGB_img -= 128
    RD_points = [(0, 100000, component, 256)]
    #RD_points = []
    component_index = components.index(component)
    for Q_step in Q_steps:
        component_img = RGB_img[..., component_index]
        dequantized_component_img, k = quantizer.quan_dequan(component_img, Q_step)
        k = (k + 128).astype(np.uint8)
        BPP = common.bits_per_gray_pixel(k, component + str(Q_step) + '_')
        RMSE = distortion.RMSE(component_img, dequantized_component_img)
        # We can measure the distortion in the component's domain because it is additive
        RD_points.append((BPP, RMSE, component, Q_step))
        print(f"Q_step={Q_step:>3}, rate={BPP:>.3f} bits/pixel, distortion={RMSE}")
    return RD_points

def RD_curve_(RGB_img, Q_steps, component, components, quantizer):
    RGB_img -= 128
    _RGB_img = RGB.copy()
    RD_points = []
    component_index = components.index(component)
    for Q_step in Q_steps:
        dQ_img, k_img = quantizer.quan_dequan(_RGB_img, Q_step)
        k = (k + 128).astype(np.uint8)
        BPP = common.bits_per_color_pixel(k_img, component + str(Q_step) + '_')
        RMSE = distortion.RMSE(RGB_img, dQ_img)
        RD_points.append((BPP, RMSE, component, Q_step))
        print(f"Q_step={Q_step:>3}, rate={BPP:>.3f} bits/pixel, distortion={RMSE}")
    return RD_points

def R_RD_curve(RGB_img, Q_steps, components, quantizer):
    return RD_curve(RGB_img, Q_steps, 'R', components, quantizer)

def G_RD_curve(RGB_img, Q_steps, components, quantizer):
    return RD_curve(RGB_img, Q_steps, 'G', components, quantizer)

def B_RD_curve(RGB_img, Q_steps, components, quantizer):
    return RD_curve(RGB_img, Q_steps, 'B', components, quantizer)

In [ ]:
print("R component")
R_points = R_RD_curve(RGB_img, Q_steps, components, quantizer)

In [ ]:
print("G component")
G_points = G_RD_curve(RGB_img, Q_steps, components, quantizer)

In [ ]:
print("Blue component")
B_points = B_RD_curve(RGB_img, Q_steps, components, quantizer)

In [ ]:
R_points

In [ ]:
G_points

In [ ]:
B_points

In [ ]:
pylab.figure(dpi=150)
#pylab.scatter(*zip(*[(i[0], i[1]) for i in R_points]), c='r', marker='.', s=0.5, label='R')              
#pylab.scatter(*zip(*[(i[0], i[1]) for i in G_points]), c='g', marker='.', s=0.5, label='G')              
#pylab.scatter(*zip(*[(i[0], i[1]) for i in B_points]), c='b', marker='.', s=0.5, label='B')              
pylab.plot(*zip(*[(i[0], i[1]) for i in R_points[1:]]), c='r', marker='.', label=f"R", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in G_points[1:]]), c='g', marker='.', label=f"G", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in B_points[1:]]), c='b', marker='.', label=f"B", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in R_points_2]), c='r', marker='x', label=f"R ({quantizer_2.name})", linestyle="dotted")              
#pylab.plot(*zip(*[(i[0], i[1]) for i in G_points_2]), c='g', marker='x', label=f"G ({quantizer_2.name})", linestyle="dotted")
#pylab.plot(*zip(*[(i[0], i[1]) for i in B_points_2]), c='b', marker='x', label=f"B ({quantizer_2.name})", linestyle="dotted")
pylab.title(f"RD contribution of each component ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

### Compute the slopes

In [ ]:
def compute_slopes(RD_points):
    extended_RD_points = [(0.0, 0.0, '', -1)] + RD_points
    counter = 0
    #RD_slopes = [(9.0E9, RD_points[0])]
    #RD_slopes = [(9.0E9, RD_points[0][2], RD_points[0][3])]
    RD_slopes = []
    #RD_slopes = [(100, RD_points[0][2], RD_points[0][3])]
    #BPP_first_point = RD_points[0][0]
    #RMSE_first_point = RD_points[0][1]
    #distance_to_origin_coordinates = math.sqrt(BPP_first_point*BPP_first_point + RMSE_first_point*RMSE_first_point)
    #RD_points = [(RD_points[0][1]/RD_points[0][0], RD_points[0][2], RD_points[0][3])]
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        #print(RD_points[counter])
        #delta_BPP = BPP - extended_RD_points[counter][0]
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        #delta_RMSE = RMSE - extended_RD_points[counter][1] 
        delta_RMSE = RMSE - RD_points[counter][1] 
        #print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        #RD_slopes.append((slope, i))
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    #RD_slopes.append((0.0, (0.0, RD_points[-1][2], 1)))
    return RD_slopes

#R_slopes = common.compute_slopes(R_points)
R_slopes = compute_slopes(R_points)
#G_slopes = common.compute_slopes(G_points)
G_slopes = compute_slopes(G_points)
#B_slopes = common.compute_slopes(B_points)
B_slopes = compute_slopes(B_points)

In [ ]:
R_slopes

In [ ]:
G_slopes

In [ ]:
B_slopes

### Merge the RD slopes and sort them (by slope)

In [ ]:
all_slopes = R_slopes + G_slopes + B_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]

In [ ]:
sorted_slopes

### Build the RD curve

At this point we have a list of RD points sorted by their slopes and we have two alternatives to compute the RD curve:

1. Suppose that the distortion of the reconstructed (dequantized) image is the sum of the distortions of each component, and do the same for the bit-rate.
2. Suppose that, because we are compressing a RGB image, the bit-rate of the compressed image can be slighly smaller than the addition of the bit-rates of the compressed components, basically because we will avoid extra headers.

The second alterntive has been used because it is more accurate.

In [ ]:
# Recompute 
def get_RD_curve_sorted_by_slopes(RGB_img, sorted_slopes, components, quantizer):
    RGB_img -= 128
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image.
    #Q_steps_per_component = [128, 128, 128] # This should generate a black image.
    for i in sorted_slopes:
        k = np.empty_like(RGB_img)
        y = np.empty_like(RGB_img)
        #point = i[1]
        #component = point[2]
        component = i[1]
        #Q_step = point[3]
        Q_step = i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        #print(i, Q_steps_per_component)
        for c, QS in zip(components, Q_steps_per_component):
            y[..., components.index(c)], k[..., components.index(c)] = quantizer.quan_dequan(RGB_img[..., components.index(c)], QS)
            #print(c, QS,components.index(c))
            #for _y in range(10):
            #    for _x in range(10):
            #        print(RGB_img[_y, _x, components.index(c)], y[_y,_x, components.index(c)], end=' ')
            #    print()
        #y, k = quantizer.quan_dequan(RGB_img, Q_step)
        k = (k + 128).astype(np.uint8)
        rate = common.bits_per_color_pixel(k, str(Q_steps_per_component) + '_')
        _distortion = distortion.RMSE(RGB_img, y)
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate} bits/pixel, distortion={_distortion}")
    return points

RD_points_sorted_by_slopes = get_RD_curve_sorted_by_slopes(RGB_img, sorted_slopes, components, quantizer)

## For further comparisons

In [ ]:
print(quantizer.name)
with open(f"RGB.txt", 'w') as f:
    for item in RD_points_sorted_by_slopes:
        f.write(f"{item[0]}\t{item[1]}\n")

## Let's compare!

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_sorted_by_slopes), c='g', marker='o', label="Sorted by slope", linestyle="dashed")
pylab.plot(*zip(*same_delta_RD_points), c='m', marker='x', label="$\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}}$", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

## Conclusion

In the RGB domain, the optimal RD curve matches the constant quantization RD curve because the 3 components have the same weight in the contribution to the quality the reconstruction. However, we have more OTPs (Optimal Truncation Points) in the optimal one.

## Comparing all quantizers

In [ ]:
RGB_midrise = []
with open(f"RGB_mid-rise.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_midrise.append((float(rate), float(_distortion)))

In [ ]:
RGB_deadzone = []
with open(f"RGB_dead-zone.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_deadzone.append((float(rate), float(_distortion)))

In [ ]:
RGB_midtread = []
with open(f"RGB_mid-tread.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_midtread.append((float(rate), float(_distortion)))

In [ ]:
RGB_companded = []
with open(f"RGB_companded.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_companded.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_midrise), c='g', marker='o', label="Mid-rise", linestyle="dashed")
pylab.plot(*zip(*RGB_deadzone), c='m', marker='x', label="Dead-zone", linestyle="dotted")
pylab.plot(*zip(*RGB_midtread), c='r', marker='x', label="Mid-read", linestyle="dotted")
pylab.plot(*zip(*RGB_companded), c='b', marker='x', label="Companded", linestyle="dotted")
pylab.title("Rate/Distortion Performance ")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

## Ignore the rest ...

## Comparing all quantizers
You must have run the previous cells for the different quantizers.

In [ ]:
RGB_midrise = []
with open(f"RGB_same_mid-rise_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_midrise.append((float(rate), float(_distortion)))

In [ ]:
RGB_deadzone = []
with open(f"RGB_same_dead-zone_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_deadzone.append((float(rate), float(_distortion)))

In [ ]:
RGB_midtread = []
with open(f"RGB_same_mid-tread_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_midtread.append((float(rate), float(_distortion)))

In [ ]:
RGB_companded = []
with open(f"RGB_same_companded_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_companded.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_midrise), c='g', marker='o', label="Mid-rise", linestyle="dashed")
pylab.plot(*zip(*RGB_deadzone), c='m', marker='x', label="Dead-zone", linestyle="dotted")
pylab.plot(*zip(*RGB_midtread), c='r', marker='x', label="Mid-read", linestyle="dotted")
pylab.plot(*zip(*RGB_companded), c='b', marker='x', label="Companded", linestyle="dotted")
pylab.title("Rate/Distortion Performance ")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

The RD performance of the four quantizers in the RGB domain is similar, but the companded quantizer needs extra bits for representing the source losslessly. This is a consequence of the floating point arithmetic used in the compressor and the decompressor.

In [ ]:
def accumulate_rate(points_sorted_by_slopes):
    optimal_points = []
    accumulated_BR = 0.0
    for i in points_sorted_by_slopes:
        BPP = i[1][0]; MSE = i[1][1]
        accumulated_BR += BPP
        optimal_points.append((accumulated_BR, MSE))
    return optimal_points

optimal_points = accumulate_rate(optimal_slopes)

In [ ]:
optimal_points

## Compute slopes

In [ ]:
def compute_slopes(RD_points):
    extended_RD_points = [(0.0, 9.0E9, '', -1)] + RD_points
    counter = 0
    RD_slopes = []
    for i in RD_points:
        BPP = i[0] # Rate 
        print(RD_points[counter])
        delta_BPP = BPP - extended_RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - extended_RD_points[counter][1] 
        #print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        if delta_BPP > 0:
            slope = delta_RMSE/delta_BPP
        else:
            slope = 0
        RD_slopes.append((slope, i[2], i[3]))
        counter += 1
    return RD_slopes

RD_slopes = compute_slopes(sorted_RD_points)

In [ ]:
RD_slopes

In [ ]:
print(RD_points)

In [ ]:
with open('RGB.txt', 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
k, y = q_deq(img, 64)
show(y, "")

## RD curves of each channel

In [ ]:
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        R_frame = RGB_frame[:,:,0]
        k, dequantized_R_frame = q_deq(R_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.RMSE(R_frame, dequantized_R_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        G_frame = RGB_frame[:,:,1]
        k, dequantized_G_frame = q_deq(G_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.RMSE(G_frame, dequantized_G_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        B_frame = RGB_frame[:,:,2]
        k, dequantized_B_frame = q_deq(B_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.RMSE(B_frame, dequantized_B_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(img)
only_G_points = only_G_RD_curve(img)
only_B_points = only_B_RD_curve(img)

In [ ]:
only_B_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='Only R')              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='Only G')              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='Only B')              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

The $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$ quantization scheme is near optimal because the slope at the different quantization points is almost the same. This can be seen in the next experiment

## Testing a different quantization configuration

In [ ]:
N = 6
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<N)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<N)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(img)
only_G_points = only_G_RD_curve(img)
only_B_points = only_B_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='$\Delta_{\mathrm{R}}~\mathrm{varies},~\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='$\Delta_{\mathrm{G}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='$\Delta_{\mathrm{B}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=$' + '{}'.format(1<<N))              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

As it can be seen, the best configuration matches $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$.

Lo que hay que hacer es calcular el slope para cada OTP (Optimal Truncated Point) de cada canal, ordenarlos y trazar la curva RD. Esto nos daría la curva RD óptima.